In [1]:
from compute_lda import *

/home/raymond/anaconda3/envs/topicModeling/lib/python3.7/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


### Some functions

In [2]:
def count_country(df,name):
    cols = ['port_of_unlading','foreign_port_of_lading','place_of_receipt'
            ,'port_of_destination','foreign_port_of_destination'
           ,'consignee_address','shipper_address']
    print('total rows is {}'.format(data.shape[0]))
    for i,col in enumerate(cols):
        temp_col = data[col].str.contains(name,case=False,regex=False)
        print('{} contains {} of rows of {}'.format(col,temp_col.sum(),name))
        if i==0:
            sum_col = temp_col.copy()
        else:
            sum_col = sum_col.add(temp_col,fill_value=0)
    print('{} of rows are {}'.format(sum_col[sum_col>0].sum(),name))

In [3]:
six_countries = ['Germany','Belgium', 'Vietnam' ,'Spain', 'United Kingdom','France']

### Data Processing

In [4]:
data = read_data()

In [5]:
data.shape

(3825317, 26)

In [7]:
data = process_data(data)

/home/raymond/Research/FEIII-SHIP/topicModeling/compute_lda.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['cl_shipper_party_name'] = data['shipper_party_name'].str.translate(str.maketrans(replace_dict)).values


In [8]:
data.shape

(1315991, 29)

In [9]:
data.columns

Index(['identifier', 'port_of_unlading', 'estimated_arrival_date',
       'foreign_port_of_lading', 'record_status_indicator', 'place_of_receipt',
       'port_of_destination', 'foreign_port_of_destination',
       'actual_arrival_date', 'consignee_name', 'consignee_address',
       'consignee_contact_name', 'consignee_comm_number_qualifier',
       'consignee_comm_number', 'shipper_party_name', 'shipper_address',
       'shipper_contact_name', 'shipper_comm_number_qualifier',
       'shipper_comm_number', 'description_sequence_number', 'piece_count',
       'description_text', 'harmonized_number', 'harmonized_value',
       'harmonized_weight', 'harmonized_weight_unit', 'cl_shipper_party_name',
       'shipper_id', '6_harmonized_number'],
      dtype='object')

In [10]:
for country in six_countries:
    count_country(data,country)

total rows is 1315991
port_of_unlading contains 0 of rows of Germany
foreign_port_of_lading contains 362089 of rows of Germany
place_of_receipt contains 3276 of rows of Germany
port_of_destination contains 0 of rows of Germany
foreign_port_of_destination contains 154 of rows of Germany
consignee_address contains 47362 of rows of Germany
shipper_address contains 131178 of rows of Germany
544059 of rows are Germany
total rows is 1315991
port_of_unlading contains 0 of rows of Belgium
foreign_port_of_lading contains 268641 of rows of Belgium
place_of_receipt contains 6744 of rows of Belgium
port_of_destination contains 0 of rows of Belgium
foreign_port_of_destination contains 541 of rows of Belgium
consignee_address contains 77530 of rows of Belgium
shipper_address contains 16275 of rows of Belgium
369731 of rows are Belgium
total rows is 1315991
port_of_unlading contains 0 of rows of Vietnam
foreign_port_of_lading contains 117185 of rows of Vietnam
place_of_receipt contains 3090 of rows o

### Shipper Matching

In [11]:
shipper_matching = pd.read_csv('../shipper_matching/Enigma_Enigma_6countries.csv')

In [12]:
shipper_matching.shape

(425010, 12)

In [13]:
shipper_matching.head()

,name_score,address_score,identifier_left,identifier_right,shipper_party_name_left,shipper_party_name_right,cl_shipper_party_name_left,cl_shipper_party_name_right,shipper_address_left,shipper_address_right,harmonized_number_left,harmonized_number_right
0,0.878049,0.487179,2.018013e+10,2.018050e+12,- AS AGENTS TO SHIPPER -,AS AHENT TO SHIPPER,AS AGENTS TO SHIPPER,AS AHENT TO SHIPPER,AGILITY LOGISTICS GMBH AM WALL 128 28195 BREM...,AGILITY LOGISTICS GMBH SIEMENSSTR. 2 + 4 70825...,NaN,NaN
1,0.866667,0.969697,2.018010e+12,2.018020e+12,: ATP LOGISTICS AS,ATP LOGISTICS,ATP LOGISTICS AS,ATP LOGISTICS,"AGENT OF HECNY SHIPPING LTD , 48-343601691 ...",AS AGENT OF HECNY SHIPPING LTD 48-343601691 ...,NaN,NaN
2,0.877193,0.800000,2.018010e+12,2.020000e+12,: ATP LOGISTICS AS AGENT OF HECNY,ATP LOGISTICS AS AGENT OF,ATP LOGISTICS AS AGENT OF HECNY,ATP LOGISTICS AS AGENT OF,. 48-343601691 TEL EX 48-34360 16,HECNY SHIPPING LTD 48-343601691 TEL EX 4...,NaN,NaN
3,0.925373,0.524138,2.018010e+12,2.018012e+12,: ATP LOGISTICS AS AGENT OF HECNY,ATP LOGISTICS AS AGENT OF HECNY SHI,ATP LOGISTICS AS AGENT OF HECNY,ATP LOGISTICS AS AGENT OF HECNY SHI,. 48-343601691 TEL EX 48-34360 16,UL JURIJA GAGARINA 32A 8 00-754 WARSZAWA POLAN...,NaN,NaN
4,0.953846,0.524138,2.018010e+12,2.018010e+12,: ATP LOGISTICS AS AGENT OF HECNY,ATP LOGISTICS AS AGENT OF HECNY S,ATP LOGISTICS AS AGENT OF HECNY,ATP LOGISTICS AS AGENT OF HECNY S,. 48-343601691 TEL EX 48-34360 16,UL JURIJA GAGARINA 32A 8 00-754 WARSZAWA POLAN...,NaN,NaN


In [14]:
shipper_matching[(shipper_matching['name_score']>0.9) & (shipper_matching['address_score']>0.6)].shape

(33916, 12)